# Treinamento

## Inicialização

In [1]:
# Config Inicial
import json
import mlflow.pyfunc
import shutil
import pandas as pd
import requests
from sklearn.metrics import r2_score
import mlflow
from mlflow.tracking import MlflowClient

import os
import signal
import subprocess


### Leitura

In [2]:
# Descomentar, quando quiser carregar para o treinamento, dados originais defasados para primeira execução do treinamento
#shutil.copyfile("dataset/brasil_estado_cidade.csv", "dataset/dados_treinamento.csv")

# Carregar o conjunto de dados
df_treinamento = pd.read_csv("dataset/dados_treinamento.csv", encoding="utf-8")

# Eliminando registros com valores null
df_treinamento.dropna(inplace=True)

# float64
df_treinamento = df_treinamento.astype({col: 'float64' for col in df_treinamento.select_dtypes(include='int').columns})

df_treinamento.head()

,id,property_type,state,region,lat,lon,area_m2,price_brl,city
0,1.0,apartment,Pernambuco,Northeast,-8.134204,-34.906326,72.0,414222.98,Recife
1,2.0,apartment,Pernambuco,Northeast,-8.126664,-34.903924,136.0,848408.53,Recife
2,3.0,apartment,Pernambuco,Northeast,-8.125550,-34.907601,75.0,299438.28,Recife
3,4.0,apartment,Pernambuco,Northeast,-8.120249,-34.895920,187.0,848408.53,Recife
4,5.0,apartment,Pernambuco,Northeast,-8.142666,-34.906906,80.0,464129.36,Recife


In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")

mlflow.set_experiment("ecd15")

<Experiment: artifact_location='/app/mlruns/0', creation_time=1744907315763, experiment_id='0', last_update_time=1744907894136, lifecycle_stage='active', name='ecd15', tags={}>

### Amostragem

In [4]:
# amostra
df_sample = df_treinamento.sample(n=20, random_state=42)

x_features = df_sample.drop(["price_brl"], axis=1)  # Features
y_target = df_sample["price_brl"]  # Variável alvo

df_sample.head()

,id,property_type,state,region,lat,lon,area_m2,price_brl,city
6383,6384.0,apartment,Rio de Janeiro,Southeast,-22.921227,-43.225548,125.0,787484.34,Rio de Janeiro
9172,9173.0,apartment,Santa Catarina,South,-27.750971,-48.779408,241.0,933875.69,Santo Amaro da Imperatriz
3868,3869.0,apartment,Rio de Janeiro,Southeast,-22.954330,-43.195595,112.0,1397378.74,Rio de Janeiro
4676,4677.0,apartment,Rio de Janeiro,Southeast,-23.000669,-43.392197,106.0,998127.67,Rio de Janeiro
18133,18134.0,house,SP,southeast,-23.588417,-46.664858,350.0,4000000.00,São Paulo


In [5]:
# Gerando o JSON no formato esperado pelo MLflow Serve
dados = {"instances": x_features.to_dict(orient="records")}
json_data = json.dumps(dados)

In [6]:
previsao = requests.post(
    "http://localhost:5000/invocations",
    headers={"Content-Type": "application/json"},
    data=json_data
)
print("HTTP STATUS", previsao.status_code)

HTTP STATUS 200


In [7]:
previsao.text

'{"predictions": [978233.0625, 906950.25, 1373972.5, 1051356.75, 1151326.0, 669886.3125, 1369107.5, 1067672.75, 358558.21875, 1130626.0, 673395.375, 715957.8125, 689171.1875, 1025426.875, 687039.875, 1130626.0, 1029650.625, 959656.5, 849469.125, 1066053.125]}'

### Carregar e identificar o melhor modelo treinado

In [8]:

def carregar_modelo_com_melhor_r2(experimento_nome):
    client = MlflowClient()

    # Pega o experimento
    experimento = client.get_experiment_by_name(experimento_nome)
    if not experimento:
        raise ValueError(f"Experimento '{experimento_nome}' não encontrado.")
    
    experiment_id = experimento.experiment_id

    melhor_modelo = {
        "r2": float('-inf'),
        "model_uri": None,
        "nome_modelo": None,
        "versao_modelo": None,
        "run_id": None
    }

    modelo_api = {
        "r2": float('-inf'),
        "model_uri": None,
        "nome_modelo": None,
        "versao_modelo": None,
        "run_id": None
    }

    # Pega todas as runs ordenadas por R² (decrescente)
    runs = client.search_runs(
        experiment_ids=[experiment_id],
        order_by=["metrics.r2 DESC"]
    )

    modelos_registrados = client.search_registered_models()

    

    for modelo in modelos_registrados:
        
        nome_modelo = modelo.name

        for versao in modelo.latest_versions:
            # Busca informações completas da versão
            versoes = client.search_model_versions(f"name='{nome_modelo}'")
            for v in versoes:
                run = client.get_run(v.run_id)

                # Verifica se pertence ao experimento desejado
                if run.info.experiment_id != experiment_id:
                    continue

                r2 = run.data.metrics.get("r2")

                source_path = v.source
                artifact_path = source_path.split("/")[-1]
                model_uri = f"runs:/{v.run_id}/{artifact_path}"

                print(f"Modelo '{modelo.name}' versão {v.version} da run {v.run_id} com R² = {r2}")

                if v.current_stage == "Production":
                    modelo_api.update({
                        "r2": r2,
                        "model_uri": model_uri,
                        "nome_modelo": nome_modelo,
                        "versao_modelo": v.version,
                        "run_id": v.run_id
                    })
                
                if r2 is not None and r2 > melhor_modelo["r2"] and v.current_stage != "Archived":
                    melhor_modelo.update({
                        "r2": r2,
                        "model_uri": model_uri,
                        "nome_modelo": nome_modelo,
                        "versao_modelo": v.version,
                        "run_id": v.run_id
                    })
    
    
    print(f"Melhor modelo '{melhor_modelo['nome_modelo']}' versão {melhor_modelo['versao_modelo']} da run {melhor_modelo['run_id']} com R² = {melhor_modelo['r2']:.4f}")

    print(f"Modelo API '{modelo_api['nome_modelo']}' versão {modelo_api['versao_modelo']} da run {modelo_api['run_id']} com R² = {modelo_api['r2']:.4f}")

    modelo_carregado = mlflow.pyfunc.load_model(melhor_modelo["model_uri"])
    
    return modelo_carregado, melhor_modelo, modelo_api
    

# Carregando modelo com melhor R2 
loaded_model, melhor_modelo, modelo_api = carregar_modelo_com_melhor_r2("ecd15")

if not melhor_modelo["model_uri"]:
    print("Nenhum modelo em 'Staging' encontrado para esse experimento.")

existe_modelo_api = True
if not modelo_api["model_uri"]:
    existe_modelo_api = False
    print("Nenhum modelo em 'Production' encontrado para esse experimento.")

melhor_modelo, modelo_api


Modelo 'decision_tree_model' versão 18 da run 23f68634125f4e6aba26569957016792 com R² = 0.7569166994450989
Modelo 'decision_tree_model' versão 17 da run 150cfef45bfa41cd9ecdafe4ba8ad274 com R² = 0.7667059447759692
Modelo 'decision_tree_model' versão 16 da run 6ef5e3a3072a42ea8b82930fbea43f7f com R² = 0.8182087518611222
Modelo 'decision_tree_model' versão 15 da run ed05ebdb1134422389b41cd0e85464fd com R² = 0.745466232475294
Modelo 'decision_tree_model' versão 14 da run f8cef16acb504893a95847cebb0a50f6 com R² = 0.7797718609172126
Modelo 'decision_tree_model' versão 13 da run 5afc011a18654fbe98cd94e534a356c5 com R² = 0.8001423748708438
Modelo 'decision_tree_model' versão 12 da run 711a41697187411f9a14ab5251bc3465 com R² = 0.7090054436900005
Modelo 'decision_tree_model' versão 11 da run dae3aeec7f904255888e7111ef45cb36 com R² = 0.725827778319228
Modelo 'decision_tree_model' versão 10 da run 54f38693af7445df98991da0e161ff03 com R² = 0.7670971810981704
Modelo 'decision_tree_model' versão 9 d

Modelo 'decision_tree_model' versão 8 da run e2bfe8f204a6443b948b43d1d1e9fb3e com R² = 0.4501405374386255
Modelo 'decision_tree_model' versão 7 da run 3b26914ee43a42a1b555950bab92eff0 com R² = 0.4662427582353449
Modelo 'decision_tree_model' versão 6 da run 87b82fd6df43456cbac3f5b23fc1bff6 com R² = 0.4342869398037643
Modelo 'decision_tree_model' versão 5 da run b6f5c3ba42264a0eb2a1f59bf9583e13 com R² = 0.4688968380333659
Modelo 'decision_tree_model' versão 4 da run 3acdfe766be546bca53899f8519cfe6b com R² = 0.4607357460145043
Modelo 'decision_tree_model' versão 3 da run 2c1c0982f0934b0eb0200e07dbd3daa4 com R² = 0.47475884579582217
Modelo 'decision_tree_model' versão 2 da run 9ebb52752ae84be58ef8aa5eacad1477 com R² = 0.4839914622045445
Modelo 'decision_tree_model' versão 1 da run ab47ff20b3ad490ea4e8f847603a667c com R² = 0.4643648515387715


Modelo 'gradient_boosting_model' versão 18 da run c102d3f88cc44df4a5c89c48aefef2db com R² = 0.7571880920801617
Modelo 'gradient_boosting_model' versão 17 da run 3eea98cdfe57400bbec20dc4171d7a9c com R² = 0.7596712814915327
Modelo 'gradient_boosting_model' versão 16 da run 80a5771892344126bdc266e88f83eea7 com R² = 0.7863802794605161
Modelo 'gradient_boosting_model' versão 15 da run 4b6d9d6cfddb4aaf85a393e31de2e08e com R² = 0.7616068455531215
Modelo 'gradient_boosting_model' versão 14 da run b0a6d92f684e47baa912b8395c06f371 com R² = 0.753435098460034
Modelo 'gradient_boosting_model' versão 13 da run a50aca3ee5fa452dbbc2d7d934366b16 com R² = 0.7596733339941215
Modelo 'gradient_boosting_model' versão 12 da run f347da595f9d4924b493964609c5b040 com R² = 0.7643279504584899
Modelo 'gradient_boosting_model' versão 11 da run 4cfc82e75b7b489598742b6f242696ae com R² = 0.768577748198987
Modelo 'gradient_boosting_model' versão 10 da run 22cfb978a6574be79ff2e1223011f722 com R² = 0.7754390469506066


Modelo 'gradient_boosting_model' versão 9 da run f044bb507c834d06ab5d6d82712182cf com R² = 0.6523843262185738
Modelo 'gradient_boosting_model' versão 8 da run 05ae2a6d50c747ada336fe92da0a8f27 com R² = 0.6492820401100539
Modelo 'gradient_boosting_model' versão 7 da run 1b9d123a59c24ac3b5590efa906b0709 com R² = 0.6709393751907473
Modelo 'gradient_boosting_model' versão 6 da run a777a623d06442c0bcfc3cc601f00bd9 com R² = 0.656862768494582
Modelo 'gradient_boosting_model' versão 5 da run a635b10f892b4f699f4cea8440b371ab com R² = 0.6599351198710504
Modelo 'gradient_boosting_model' versão 4 da run 54867fd3396248e29e04dc85bd95d293 com R² = 0.6600230971249696
Modelo 'gradient_boosting_model' versão 3 da run 3249067b29234dcda01cd072eb4ff7c3 com R² = 0.6796071604125762
Modelo 'gradient_boosting_model' versão 2 da run 8ae7fe8a8ed9465e89ae3affdfc01647 com R² = 0.6872079997884422
Modelo 'gradient_boosting_model' versão 1 da run 9797126887fc418e80976a54d4972001 com R² = 0.6753892857373056


Modelo 'linear_regression_model' versão 18 da run 726af900e51a44de9211d882c2805224 com R² = -38.837751174255274
Modelo 'linear_regression_model' versão 17 da run af1321eb7f8f4a55b34aed9672082920 com R² = -75.20402755500736
Modelo 'linear_regression_model' versão 16 da run 279392b71b244e45ab2d4d6fb116e291 com R² = -53.61064994753689
Modelo 'linear_regression_model' versão 15 da run 721d6209a4a94186b03d42a674b772eb com R² = 0.3329851214196373
Modelo 'linear_regression_model' versão 14 da run 118ac6ede0804befa9c4357393320c0c com R² = 0.33032526020028585
Modelo 'linear_regression_model' versão 13 da run b13b7d38438a47ff965f205cf5ce4d9c com R² = 0.3195849483844625
Modelo 'linear_regression_model' versão 12 da run ec8c6f3833594c3e9693829090a0693f com R² = -42.622918299380565
Modelo 'linear_regression_model' versão 11 da run e509cd776ec14a98a516660f0981ab66 com R² = -86.29611390384845
Modelo 'linear_regression_model' versão 10 da run fcbf9e53346b4f7c8337d96ade1a5759 com R² = 0.338116621627624

...

Modelo 'xgboost_model' versão 18 da run 348a9d5f9f3247ed83bf175a45fc366d com R² = 0.8427781011631683
Modelo 'xgboost_model' versão 17 da run 3ce29d9937d34f4b9f32aa4440dfe6d5 com R² = 0.8349147608600651
Modelo 'xgboost_model' versão 16 da run b0962634469b496982673e420f041365 com R² = 0.863893477059456
Modelo 'xgboost_model' versão 15 da run 9b2cb118c1c44b0295166cade8ddc9a5 com R² = 0.8150327499967401
Modelo 'xgboost_model' versão 14 da run f679b380c136454ea1b73f44199ed01d com R² = 0.8359118294814103
Modelo 'xgboost_model' versão 13 da run a13e6a5ed49c4854af31b59d46bf3bf9 com R² = 0.8344277104028837
Modelo 'xgboost_model' versão 12 da run 992cd26a63df4a54b3fae36e42dbf1f1 com R² = 0.8376653626985738
Modelo 'xgboost_model' versão 11 da run c3a295ce39f9416ba680aa97c5769966 com R² = 0.8349100161979262
Modelo 'xgboost_model' versão 10 da run afa5e381b8484494b672b388043a6385 com R² = 0.837036426485852


Modelo 'xgboost_model' versão 1 da run f1648a50cd9740c2a286f2103a29b1c8 com R² = 0.6904415768747783
Modelo 'xgboost_model' versão 2 da run a957f24328194282bd9c523bcfb7cb38 com R² = 0.7128160768989399
Modelo 'xgboost_model' versão 3 da run ba80e3d8f13744468ba5963965a53b67 com R² = 0.7040234028906887
Modelo 'xgboost_model' versão 4 da run 21385a91261642d6bbcb8f01bf3740c2 com R² = 0.7026213088601732
Modelo 'xgboost_model' versão 5 da run 9d822dcb2b5d4c7ab1bb29684b99494f com R² = 0.6980056141375233
Modelo 'xgboost_model' versão 6 da run ee5d37d527554e0fbdbfad283979d1c2 com R² = 0.6883462328636021
Modelo 'xgboost_model' versão 8 da run 471853b24301459a9edbeb08a3f3897d com R² = 0.6860089789442159
Modelo 'xgboost_model' versão 9 da run e8ffb7b6d6124263a2fb549e0d55c3e0 com R² = 0.6876703524937664
Modelo 'xgboost_model' versão 7 da run 8cc1b3f756e34b03bbc93c19aace7a92 com R² = 0.7165948434149507


Modelo 'xgboost_model' versão 18 da run 348a9d5f9f3247ed83bf175a45fc366d com R² = 0.8427781011631683
Modelo 'xgboost_model' versão 17 da run 3ce29d9937d34f4b9f32aa4440dfe6d5 com R² = 0.8349147608600651
Modelo 'xgboost_model' versão 16 da run b0962634469b496982673e420f041365 com R² = 0.863893477059456
Modelo 'xgboost_model' versão 15 da run 9b2cb118c1c44b0295166cade8ddc9a5 com R² = 0.8150327499967401
Modelo 'xgboost_model' versão 14 da run f679b380c136454ea1b73f44199ed01d com R² = 0.8359118294814103
Modelo 'xgboost_model' versão 13 da run a13e6a5ed49c4854af31b59d46bf3bf9 com R² = 0.8344277104028837
Modelo 'xgboost_model' versão 12 da run 992cd26a63df4a54b3fae36e42dbf1f1 com R² = 0.8376653626985738
Modelo 'xgboost_model' versão 11 da run c3a295ce39f9416ba680aa97c5769966 com R² = 0.8349100161979262
Modelo 'xgboost_model' versão 10 da run afa5e381b8484494b672b388043a6385 com R² = 0.837036426485852


Modelo 'xgboost_model' versão 1 da run f1648a50cd9740c2a286f2103a29b1c8 com R² = 0.6904415768747783
Modelo 'xgboost_model' versão 2 da run a957f24328194282bd9c523bcfb7cb38 com R² = 0.7128160768989399
Modelo 'xgboost_model' versão 3 da run ba80e3d8f13744468ba5963965a53b67 com R² = 0.7040234028906887
Modelo 'xgboost_model' versão 4 da run 21385a91261642d6bbcb8f01bf3740c2 com R² = 0.7026213088601732
Modelo 'xgboost_model' versão 5 da run 9d822dcb2b5d4c7ab1bb29684b99494f com R² = 0.6980056141375233
Modelo 'xgboost_model' versão 6 da run ee5d37d527554e0fbdbfad283979d1c2 com R² = 0.6883462328636021
Modelo 'xgboost_model' versão 8 da run 471853b24301459a9edbeb08a3f3897d com R² = 0.6860089789442159
Modelo 'xgboost_model' versão 9 da run e8ffb7b6d6124263a2fb549e0d55c3e0 com R² = 0.6876703524937664
Modelo 'xgboost_model' versão 7 da run 8cc1b3f756e34b03bbc93c19aace7a92 com R² = 0.7165948434149507
Melhor modelo 'random_forest_model' versão 16 da run 492123208a2d4930b005ed332f2d1abb com R² = 0.890

({'r2': 0.8904461535922401,
  'model_uri': 'runs:/492123208a2d4930b005ed332f2d1abb/random_forest_model',
  'nome_modelo': 'random_forest_model',
  'versao_modelo': 16,
  'run_id': '492123208a2d4930b005ed332f2d1abb'},
 {'r2': 0.7165948434149507,
  'model_uri': 'runs:/8cc1b3f756e34b03bbc93c19aace7a92/xgboost_model',
  'nome_modelo': 'xgboost_model',
  'versao_modelo': 7,
  'run_id': '8cc1b3f756e34b03bbc93c19aace7a92'})

## Predição de melhor modelo carregado x modelo API

In [9]:
# Fazer a previsão
predictions = loaded_model.predict(x_features)
# Adicionar as previsões ao DataFrame
    
df_sample["predicao_melhor_modelo"] = predictions

# converter previsões para o mesmo tipo da variável alvo
df_sample["predicao_melhor_modelo"] = df_sample["predicao_melhor_modelo"].astype(float).round(2)

colunas_exibidas = ["price_brl", "predicao_melhor_modelo"]

if existe_modelo_api:
    previsao_json = json.loads(previsao.text)

    # Extrair as previsões do JSON e adicioná-las ao DataFrame
    df_sample["predicao_modelo_api"] = [round(pred, 2) for pred in previsao_json["predictions"]]

    colunas_exibidas = ["price_brl", "predicao_modelo_api", "predicao_melhor_modelo"]

df_sample[colunas_exibidas]

,price_brl,predicao_modelo_api,predicao_melhor_modelo
6383,787484.34,978233.06,844436.32
9172,933875.69,906950.25,1100024.97
3868,1397378.74,1373972.50,1401079.65
4676,998127.67,1051356.75,1009376.21
18133,4000000.00,1151326.00,5049400.00
13351,1890000.00,669886.31,1792180.00
17172,1700000.00,1369107.50,1626377.39
17379,1300000.00,1067672.75,1281245.14
5818,289457.01,358558.22,309616.08
20470,3750000.00,1130626.00,3660319.60


### Comparação R² Score dos modelos

In [10]:
# Calcular R²
r2_melhor_modelo = r2_score(df_sample["price_brl"], df_sample["predicao_melhor_modelo"])

if existe_modelo_api:
    r2_api = r2_score(df_sample["price_brl"], df_sample["predicao_modelo_api"])
    print(f"R² API: {r2_api:.4f} R² Melhor Modelo: {r2_melhor_modelo:.4f}")
else:
    print(f"R² Melhor Modelo: {r2_melhor_modelo:.4f}")

R² API: 0.9782 R² Melhor Modelo: 0.9782


### Promover melhor modelo para Produção

In [11]:
def parar_modelo_mlflow1():
    try:
        resultado = subprocess.check_output("lsof -i :5000 -sTCP:LISTEN -nP", shell=True)
        linhas = resultado.decode().strip().split("\n")[1:]  # ignora cabeçalho
        for linha in linhas:
            if "mlflow" in linha.lower():
                pid = int(linha.split()[1])
                os.kill(pid, signal.SIGTERM)
                print(f"Servidor MLflow (PID {pid}) parado com sucesso.")
            else:
                print("⚠️ Porta 5000 ocupada por outro processo que não é o MLflow. Ignorado.")
    except subprocess.CalledProcessError:
        print("Nenhum processo ouvindo na porta 5000 foi encontrado.")

def promover_modelo_para_producao(modelo):
    client = MlflowClient()

    nome_modelo = modelo["nome_modelo"]
    versao_modelo = modelo["versao_modelo"]

    # Agora vamos arquivar os que ainda estão em Staging
    todos_modelos = client.search_model_versions("name LIKE '%_model'")
    modelos_ativos = [v for v in todos_modelos if v.current_stage != "Archived"]

    
    for v in modelos_ativos:
        client.transition_model_version_stage(
             name=v.name,
             version=v.version,
             stage="Archived"
        )
        print(f"Modelo { v.name } Versão {v.version} arquivado.")

    # Promover o modelo selecionado
    client.transition_model_version_stage(
        name=nome_modelo,
        version=versao_modelo,
        stage="Production",
        archive_existing_versions=True  # arquiva automaticamente os outros em Production
    )
    print(f"Modelo '{nome_modelo}' versão {versao_modelo} promovido para 'Production'.")


def servir_melhor_modelo(experimento_nome, modelo):
    client = MlflowClient()
    
    # Parar modelo atual
    os.system("bash parar_mlflow_serve.sh")

    model_uri = modelo["model_uri"]

    promover_modelo_para_producao(modelo)

    #Tentativas de tentar parar o servidor e subir via notebook não funcionaram corretamente, deixando a utilização lenta.
    #print("É necessário parar e inicializar o docker novamente para carregar novo modelo na API automaticamente")

    # Reinicializar api com novo modelo
    os.system(f"bash reiniciar_mlflow_serve.sh {model_uri}")

servir_melhor_modelo("ecd15", melhor_modelo)

O processo na porta 5000 foi encerrado com sucesso.


Modelo xgboost_model Versão 18 arquivado.
Modelo gradient_boosting_model Versão 18 arquivado.


/tmp/ipykernel_1620/547996578.py:27: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Modelo random_forest_model Versão 18 arquivado.


Modelo linear_regression_model Versão 18 arquivado.
Modelo decision_tree_model Versão 18 arquivado.


...

Modelo decision_tree_model Versão 3 arquivado.
Modelo gradient_boosting_model Versão 2 arquivado.
Modelo random_forest_model Versão 2 arquivado.


Modelo linear_regression_model Versão 2 arquivado.
Modelo decision_tree_model Versão 2 arquivado.
Modelo gradient_boosting_model Versão 1 arquivado.


Modelo random_forest_model Versão 1 arquivado.
Modelo linear_regression_model Versão 1 arquivado.
Modelo decision_tree_model Versão 1 arquivado.


Modelo 'random_forest_model' versão 16 promovido para 'Production'.


/tmp/ipykernel_1620/547996578.py:35: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
